- first filtering data with sleeping time removed 
- select best data which the action lead to stable glucose for joint-training model
- for joint-training model and reinforcement learning: decide insulin, dietary, physicial activity categorizations for human-managed and machine-managed scenarios

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import shap
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [2]:
df1 = pd.read_csv("preprocessed_data_v2.csv")
df1

,index,Device_Code,age,Hemoglobin_A1c,height_m,weight_kg,BMI,obesity_category,sex_F,sex_M,race_AMERICAN INDIAN/ALASKAN NATIVE,race_ASIAN,race_BLACK/AFRICAN AMERICAN,race_MULTIPLE,race_UNKNOWN,race_WHITE,ethnicity_Hispanic or Latino,ethnicity_Not Hispanic or Latino,ethnicity_UNKNOWN,insulin_delivery_MULTIPLE DAILY INJECTIONS,insulin_delivery_PUMP OR CLOSED LOOP,measure_sequence,Physicial_Activity,Calories_Burned,Heart_Rate,Insulin,Dietary_Total_Carbohydrate,Glucose,month,day,day_name,hour,hour_name
0,100,0,14.0,10.9,1.7526,70.306760,22.889245,1,0,1,0,1,0,0,0,0,0,1,0,1,0,1.0,0,0.0,82.70864,0.0,0.0,125.666667,8,10,2,1,1
1,100,0,14.0,10.9,1.7526,70.306760,22.889245,1,0,1,0,1,0,0,0,0,0,1,0,1,0,2.0,0,0.0,82.70864,0.0,0.0,125.000000,8,10,2,2,1
2,100,0,14.0,10.9,1.7526,70.306760,22.889245,1,0,1,0,1,0,0,0,0,0,1,0,1,0,3.0,0,0.0,82.70864,0.0,0.0,159.654188,8,10,2,2,1
3,100,0,14.0,10.9,1.7526,70.306760,22.889245,1,0,1,0,1,0,0,0,0,0,1,0,1,0,4.0,0,0.0,82.70864,0.0,0.0,125.000000,8,10,2,2,1
4,100,0,14.0,10.9,1.7526,70.306760,22.889245,1,0,1,0,1,0,0,0,0,0,1,0,1,0,5.0,0,0.0,82.70864,0.0,0.0,130.333333,8,10,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264592,99,2,12.0,6.3,1.6256,57.606184,21.799259,1,1,0,0,0,0,0,0,1,0,1,0,0,1,956.0,0,0.0,82.70864,0.0,0.0,129.666667,10,15,5,22,8
264593,99,2,12.0,6.3,1.6256,57.606184,21.799259,1,1,0,0,0,0,0,0,1,0,1,0,0,1,957.0,0,0.0,82.70864,0.0,0.0,143.000000,10,15,5,23,8
264594,99,2,12.0,6.3,1.6256,57.606184,21.799259,1,1,0,0,0,0,0,0,1,0,1,0,0,1,958.0,0,0.0,82.70864,0.0,0.0,132.000000,10,15,5,23,8
264595,99,2,12.0,6.3,1.6256,57.606184,21.799259,1,1,0,0,0,0,0,0,1,0,1,0,0,1,959.0,0,0.0,82.70864,0.0,0.0,125.000000,10,15,5,23,8


In [ ]:
df1['']

### removed data during sleeping time 

In [3]:
# # Define the values you want to remove
# values_to_remove = [23, 0, 1, 2, 3, 4, 5, 6]

# # Filter the dataframe to remove rows with the specified values
# df2 = df1[~df1['hour'].isin(values_to_remove)]

# df2['hour'].unique()

In [4]:
df2 = df1.copy()

### different categorizations of insulin, dietary values for human-managed and machine-managed

In [5]:
# split data

df_human = df2[df2['insulin_delivery_MULTIPLE DAILY INJECTIONS']==1]
df_pump = df2[df2['insulin_delivery_MULTIPLE DAILY INJECTIONS']!=1]

#### 1. For human

#### 1.1. insulin

In [6]:
iu_col_h = df_human['Insulin']
iu_col_h.describe()

count    41471.000000
mean         0.450399
std          3.622547
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        167.500000
Name: Insulin, dtype: float64

In [7]:
iu_col_h.quantile(q=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

0.1      0.0
0.2      0.0
0.3      0.0
0.4      0.0
0.5      0.0
0.6      0.0
0.7      0.0
0.8      0.0
0.9      0.0
1.0    167.5
Name: Insulin, dtype: float64

In [8]:
def percent1(iu_col_h):
    filtered_data = iu_col_h[iu_col_h == 0]
    print('=0:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))
    
    filtered_data = iu_col_h[(iu_col_h > 0) & (iu_col_h <= 1)]
    print('0<v<=1:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))
    
    filtered_data = iu_col_h[(iu_col_h > 0) & (iu_col_h <= 5)]
    print('0<v<=5:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))    
    
    filtered_data = iu_col_h[(iu_col_h > 1) & (iu_col_h <= 10)]
    print('1<v<=10:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))

    filtered_data = iu_col_h[(iu_col_h > 5) & (iu_col_h <= 10)]
    print('5<v<=10:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))
    
    filtered_data = iu_col_h[iu_col_h > 10]
    print('>10:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))

percent1(iu_col_h)

=0: 96.3
0<v<=1: 0.21
0<v<=5: 1.39
1<v<=10: 2.09
5<v<=10: 0.91
>10: 1.4


In [9]:
# insulin range:
# 1. 0
# 2. 0-5
# 3. 5-10
# 4. > 10

df_human['Insulin_Category'] = pd.cut(
    df_human['Insulin'],
    bins=[-0.1, 0, 5, 10, float('inf')],
    labels=['0', '0-5', '5-10', '> 10'],
    include_lowest=True)

/tmp/ipykernel_1213443/1148506760.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_human['Insulin_Category'] = pd.cut(


In [10]:
value_counts = df_human['Insulin_Category'].value_counts()
(value_counts / len(df_human)) * 100

Insulin_Category
0       96.303441
> 10     1.396156
0-5      1.388922
5-10     0.911480
Name: count, dtype: float64

In [11]:
insulin_mapping = {
    '0': 0,
    '0-5': 1,
    '5-10': 2,
    '> 10': 3}

df_human['Insulin_Category'] = df_human['Insulin_Category'].map(insulin_mapping)
value_counts = df_human['Insulin_Category'].value_counts()
(value_counts / len(df_human)) * 100

/tmp/ipykernel_1213443/2031957740.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_human['Insulin_Category'] = df_human['Insulin_Category'].map(insulin_mapping)


Insulin_Category
0    96.303441
3     1.396156
1     1.388922
2     0.911480
Name: count, dtype: float64

#### 1.2. dietary

In [12]:
cho_col_h = df_human['Dietary_Total_Carbohydrate']
cho_col_h.describe()

count    41471.000000
mean         1.099433
std         11.009018
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        300.000000
Name: Dietary_Total_Carbohydrate, dtype: float64

In [13]:
cho_col_h.quantile(q=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

0.1      0.0
0.2      0.0
0.3      0.0
0.4      0.0
0.5      0.0
0.6      0.0
0.7      0.0
0.8      0.0
0.9      0.0
1.0    300.0
Name: Dietary_Total_Carbohydrate, dtype: float64

In [14]:
def percent1(iu_col_h):
    filtered_data = iu_col_h[iu_col_h == 0]
    print('=0:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))
    
    filtered_data = iu_col_h[(iu_col_h > 0) & (iu_col_h <= 10)]
    print('0<v<=10:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))    

    filtered_data = iu_col_h[(iu_col_h > 0) & (iu_col_h <= 50)]
    print('0<v<=50:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))  
    
    filtered_data = iu_col_h[iu_col_h > 10]
    print('>10:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))
    
    filtered_data = iu_col_h[iu_col_h > 50]
    print('>50:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))
    
    filtered_data = iu_col_h[(iu_col_h > 50) & (iu_col_h <= 100)]
    print('50<v<=100:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))   
    
    filtered_data = iu_col_h[iu_col_h > 100]
    print('>100:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))    
percent1(cho_col_h)

=0: 98.46
0<v<=10: 0.09
0<v<=50: 0.71
>10: 1.45
>50: 0.84
50<v<=100: 0.46
>100: 0.37


In [15]:
# dietary range:
# 1. 0
# 2. 0-50
# 3. 50-100
# 4. > 100

df_human['Dietary_Category'] = pd.cut(
    df_human['Dietary_Total_Carbohydrate'],
    bins=[-0.1, 0, 50, float('inf')],
    labels=['0', '0-50', '> 50'],
    include_lowest=True)

/tmp/ipykernel_1213443/1608964366.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_human['Dietary_Category'] = pd.cut(


In [16]:
value_counts = df_human['Dietary_Category'].value_counts()
(value_counts / len(df_human)) * 100

Dietary_Category
0       98.456753
> 50     0.836729
0-50     0.706518
Name: count, dtype: float64

In [17]:
dietary_mapping = {
    '0': 0,
    '0-50': 1,
    '> 50': 2}

df_human['Dietary_Category'] = df_human['Dietary_Category'].map(dietary_mapping)
value_counts = df_human['Dietary_Category'].value_counts()
(value_counts / len(df_human)) * 100

/tmp/ipykernel_1213443/3179353546.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_human['Dietary_Category'] = df_human['Dietary_Category'].map(dietary_mapping)


Dietary_Category
0    98.456753
2     0.836729
1     0.706518
Name: count, dtype: float64

In [18]:
df_human.to_csv('preprocessed_data_v3_human.csv', index=False)

In [19]:
df_human

,index,Device_Code,age,Hemoglobin_A1c,height_m,weight_kg,BMI,obesity_category,sex_F,sex_M,race_AMERICAN INDIAN/ALASKAN NATIVE,race_ASIAN,race_BLACK/AFRICAN AMERICAN,race_MULTIPLE,race_UNKNOWN,race_WHITE,ethnicity_Hispanic or Latino,ethnicity_Not Hispanic or Latino,ethnicity_UNKNOWN,insulin_delivery_MULTIPLE DAILY INJECTIONS,insulin_delivery_PUMP OR CLOSED LOOP,measure_sequence,Physicial_Activity,Calories_Burned,Heart_Rate,Insulin,Dietary_Total_Carbohydrate,Glucose,month,day,day_name,hour,hour_name,Insulin_Category,Dietary_Category
0,100,0,14.0,10.9,1.7526,70.30676,22.889245,1,0,1,0,1,0,0,0,0,0,1,0,1,0,1.0,0,0.0,82.70864,0.0,0.0,125.666667,8,10,2,1,1,0,0
1,100,0,14.0,10.9,1.7526,70.30676,22.889245,1,0,1,0,1,0,0,0,0,0,1,0,1,0,2.0,0,0.0,82.70864,0.0,0.0,125.000000,8,10,2,2,1,0,0
2,100,0,14.0,10.9,1.7526,70.30676,22.889245,1,0,1,0,1,0,0,0,0,0,1,0,1,0,3.0,0,0.0,82.70864,0.0,0.0,159.654188,8,10,2,2,1,0,0
3,100,0,14.0,10.9,1.7526,70.30676,22.889245,1,0,1,0,1,0,0,0,0,0,1,0,1,0,4.0,0,0.0,82.70864,0.0,0.0,125.000000,8,10,2,2,1,0,0
4,100,0,14.0,10.9,1.7526,70.30676,22.889245,1,0,1,0,1,0,0,0,0,0,1,0,1,0,5.0,0,0.0,82.70864,0.0,0.0,130.333333,8,10,2,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257887,86,0,12.0,7.9,1.6510,54.43104,19.968824,1,0,1,0,0,0,0,0,1,0,1,0,1,0,917.0,0,0.0,82.70864,0.0,0.0,182.666667,6,10,4,22,8,0,0
257888,86,0,12.0,7.9,1.6510,54.43104,19.968824,1,0,1,0,0,0,0,0,1,0,1,0,1,0,918.0,0,0.0,82.70864,0.0,0.0,182.333333,6,10,4,23,8,0,0
257889,86,0,12.0,7.9,1.6510,54.43104,19.968824,1,0,1,0,0,0,0,0,1,0,1,0,1,0,919.0,0,0.0,82.70864,0.0,0.0,183.000000,6,10,4,23,8,0,0
257890,86,0,12.0,7.9,1.6510,54.43104,19.968824,1,0,1,0,0,0,0,0,1,0,1,0,1,0,920.0,0,0.0,82.70864,0.0,0.0,183.666667,6,10,4,23,8,0,0


In [42]:
value_counts = df_human['Physicial_Activity'].value_counts()
(value_counts / len(df_human2)) * 100

Physicial_Activity
0    99.556316
1     0.443684
Name: count, dtype: float64

In [20]:
df_human2 = df_human[['index','hour','Dietary_Category']]
df_human2

,index,hour,Dietary_Category
0,100,1,0
1,100,2,0
2,100,2,0
3,100,2,0
4,100,2,0
...,...,...,...
257887,86,22,0
257888,86,23,0
257889,86,23,0
257890,86,23,0


In [21]:
df_human2['hour'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23,  0])

In [22]:
# define the night‐time hours
night_hours = [23, 0, 1, 2, 3, 4, 5, 6]

# filter the dataframe
night_df = df_human2[df_human2['hour'].isin(night_hours)]

# get the percentage distribution
percentages = night_df['Dietary_Category'] \
    .value_counts(normalize=True) \
    .mul(100) \
    .sort_index()

print(percentages)

Dietary_Category
0    99.694989
1     0.167030
2     0.137981
Name: proportion, dtype: float64


In [23]:
print(len(night_df))
print(len(night_df['index'].unique()))

13770
37


#### 2. For pumps

#### 2.1. insulin

In [24]:
iu_col_p = df_pump['Insulin']
iu_col_p.describe()

count    223126.000000
mean          0.550651
std           1.898219
min           0.000000
25%           0.000000
50%           0.000000
75%           0.282000
max          49.200000
Name: Insulin, dtype: float64

In [25]:
iu_col_p.quantile(q=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

0.1     0.000
0.2     0.000
0.3     0.000
0.4     0.000
0.5     0.000
0.6     0.051
0.7     0.195
0.8     0.413
0.9     1.070
1.0    49.200
Name: Insulin, dtype: float64

In [26]:
def percent1(iu_col_h):
    filtered_data = iu_col_h[iu_col_h == 0]
    print('=0:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))
    
    filtered_data = iu_col_h[(iu_col_h > 0) & (iu_col_h <= 1)]
    print('0<v<=1:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))

    filtered_data = iu_col_h[(iu_col_h > 0) & (iu_col_h <= 0.1)]
    print('0<v<=0.1:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))

    filtered_data = iu_col_h[(iu_col_h > 0) & (iu_col_h <= 0.5)]
    print('0<v<=0.5:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))

    filtered_data = iu_col_h[(iu_col_h > 0.1) & (iu_col_h <= 1)]
    print('0.1<v<=1:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2)) 
    
    filtered_data = iu_col_h[(iu_col_h > 0.5) & (iu_col_h <= 1)]
    print('0.5<v<=1:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))  
    
    filtered_data = iu_col_h[(iu_col_h > 0) & (iu_col_h <= 5)]
    print('0<v<=5:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))    

    filtered_data = iu_col_h[(iu_col_h > 1) & (iu_col_h <= 5)]
    print('1<v<=5:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))
    
    filtered_data = iu_col_h[(iu_col_h > 1) & (iu_col_h <= 10)]
    print('1<v<=10:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))

    filtered_data = iu_col_h[iu_col_h > 1]
    print('>1:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))
    
    filtered_data = iu_col_h[iu_col_h > 5]
    print('>5:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))
    
    filtered_data = iu_col_h[(iu_col_h > 5) & (iu_col_h <= 10)]
    print('5<v<=10:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))
    
    filtered_data = iu_col_h[iu_col_h > 10]
    print('>10:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))

percent1(iu_col_p)

=0: 54.18
0<v<=1: 35.37
0<v<=0.1: 9.65
0<v<=0.5: 28.21
0.1<v<=1: 25.71
0.5<v<=1: 7.16
0<v<=5: 42.99
1<v<=5: 7.62
1<v<=10: 9.57
>1: 10.45
>5: 2.83
5<v<=10: 1.95
>10: 0.88


In [27]:
# insulin range:
# 1. 0
# 2. 0-0.5
# 3. 0.5-1
# 4. > 1

df_pump['Insulin_Category'] = pd.cut(
    df_pump['Insulin'],
    bins=[-0.1, 0, 0.5, 1, float('inf')],
    labels=['0', '0-0.5', '0.5-1', '> 1'],
    include_lowest=True)

/tmp/ipykernel_1213443/4147831035.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pump['Insulin_Category'] = pd.cut(


In [28]:
value_counts = df_pump['Insulin_Category'].value_counts()
(value_counts / len(df_pump)) * 100

Insulin_Category
0        54.182838
0-0.5    28.208725
> 1      10.448805
0.5-1     7.159632
Name: count, dtype: float64

In [29]:
insulin_mapping = {
    '0': 0,
    '0-0.5': 1,
    '0.5-1': 2,
    '> 1': 3}

df_pump['Insulin_Category'] = df_pump['Insulin_Category'].map(insulin_mapping)
value_counts = df_pump['Insulin_Category'].value_counts()
(value_counts / len(df_pump)) * 100

/tmp/ipykernel_1213443/3488587273.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pump['Insulin_Category'] = df_pump['Insulin_Category'].map(insulin_mapping)


Insulin_Category
0    54.182838
1    28.208725
3    10.448805
2     7.159632
Name: count, dtype: float64

#### 2.2. dietary

In [30]:
cho_col_p = df_pump['Dietary_Total_Carbohydrate']
cho_col_p.describe()

count    223126.000000
mean          1.061807
std          10.914971
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max         417.390000
Name: Dietary_Total_Carbohydrate, dtype: float64

In [31]:
cho_col_p.quantile(q=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

0.1      0.00
0.2      0.00
0.3      0.00
0.4      0.00
0.5      0.00
0.6      0.00
0.7      0.00
0.8      0.00
0.9      0.00
1.0    417.39
Name: Dietary_Total_Carbohydrate, dtype: float64

In [32]:
def percent1(iu_col_h):
    filtered_data = iu_col_h[iu_col_h == 0]
    print('=0:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))
    
    filtered_data = iu_col_h[(iu_col_h > 0) & (iu_col_h <= 10)]
    print('0<v<=10:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))    

    filtered_data = iu_col_h[(iu_col_h > 0) & (iu_col_h <= 50)]
    print('0<v<=50:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))  
    
    filtered_data = iu_col_h[iu_col_h > 10]
    print('>10:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))
    
    filtered_data = iu_col_h[iu_col_h > 50]
    print('>50:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))
    
    filtered_data = iu_col_h[(iu_col_h > 50) & (iu_col_h <= 100)]
    print('50<v<=100:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))   
    
    filtered_data = iu_col_h[iu_col_h > 100]
    print('>100:', round(((len(filtered_data) / len(iu_col_h))) * 100, 2))  
    
percent1(cho_col_p)

=0: 98.55
0<v<=10: 0.05
0<v<=50: 0.62
>10: 1.4
>50: 0.84
50<v<=100: 0.48
>100: 0.35


In [33]:
# dietary range:
# 1. 0
# 2. 0-50
# 3. > 50

df_pump['Dietary_Category'] = pd.cut(
    df_pump['Dietary_Total_Carbohydrate'],
    bins=[-0.1, 0, 50, float('inf')],
    labels=['0', '0-50', '> 50'],
    include_lowest=True)

/tmp/ipykernel_1213443/2768108088.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pump['Dietary_Category'] = pd.cut(


In [34]:
value_counts = df_pump['Dietary_Category'].value_counts()
(value_counts / len(df_pump)) * 100

Dietary_Category
0       98.547009
> 50     0.837643
0-50     0.615347
Name: count, dtype: float64

In [35]:
dietary_mapping = {
    '0': 0,
    '0-50': 1,
    '> 50': 2}

df_pump['Dietary_Category'] = df_pump['Dietary_Category'].map(dietary_mapping)
value_counts = df_pump['Dietary_Category'].value_counts()
(value_counts / len(df_pump)) * 100

/tmp/ipykernel_1213443/478820871.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pump['Dietary_Category'] = df_pump['Dietary_Category'].map(dietary_mapping)


Dietary_Category
0    98.547009
2     0.837643
1     0.615347
Name: count, dtype: float64

In [36]:
df_pump.to_csv('preprocessed_data_v3_pump.csv', index=False)

In [37]:
# get the percentage distribution
percentages = df_pump['Dietary_Category'] \
    .value_counts(normalize=True) \
    .mul(100) \
    .sort_index()
percentages

Dietary_Category
0    98.547009
1     0.615347
2     0.837643
Name: proportion, dtype: float64

In [43]:
value_counts = df_pump['Physicial_Activity'].value_counts()
(value_counts / len(df_pump)) * 100

Physicial_Activity
0    99.910365
1     0.089635
Name: count, dtype: float64

In [38]:
df_pump2 = df_pump[['index','hour','Dietary_Category']]
df_pump2

,index,hour,Dietary_Category
952,101,0,0
953,101,0,0
954,101,0,0
955,101,0,0
956,101,1,0
...,...,...,...
264592,99,22,0
264593,99,23,0
264594,99,23,0
264595,99,23,0


In [39]:
# define the night‐time hours
night_hours = [23, 0, 1, 2, 3, 4, 5, 6]

# filter the dataframe
night_df = df_pump2[df_pump2['hour'].isin(night_hours)]

# get the percentage distribution
percentages = night_df['Dietary_Category'] \
    .value_counts(normalize=True) \
    .mul(100) \
    .sort_index()

print(percentages)

Dietary_Category
0    99.799332
1     0.083499
2     0.117169
Name: proportion, dtype: float64


In [40]:
print(len(night_df))
print(len(night_df['index'].unique()))

74252
224
